# Preparing SDSS DR16 VACs


## Introduction

Working with the SDSS DR16 VAC files in preparation for loading into a database. There are one VAC(s) currently targeted for loading into Data Lab:

1. [DR16Q](https://www.sdss.org/dr16/data_access/value-added-catalogs/?vac_id=the-sloan-digital-sky-survey-quasar-catalogue:-sixteenth-data-release-(dr16q))
2. [ELG_CLASSIFICATION](https://www.sdss.org/dr16/data_access/value-added-catalogs/?vac_id=classification-of-intermediate-redshift-emission-line-galaxies)

All of these are conveniently stored as single FITS tables.  However, varying degrees of preprocessing may be needed.

Other resources:

* [SDSS CAS SQL definition files](https://trac.sdss.org/browser/repo/sdss/sas/trunk/sql)


## Setup

In [1]:
# Standard library
from io import BytesIO
import re
import csv
# matplotlib, etc.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager, FontProperties
import numpy as np
from astropy.io import fits
from astropy.visualization import astropy_mpl_style
from astropy.io.votable import parse_single_table
from astropy.table import Column, Table
plt.style.use(astropy_mpl_style)
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
# Data Lab
from dl import queryClient as qc, storeClient as sc
#
# Global constants.
#
release = 16
sas = 'sdss_dr{0:d}://'.format(release)
sdss_run2d = 26
boss_run2d = 'v5_13_0'
m = re.match(r'v(\d+)_(\d+)_(\d+)', boss_run2d)
N, M, P = m.groups()
boss_run2d_int = (int(N) - 5)*10000 + int(M) * 100 + int(P)
# assert matplotlib.rcParams['figure.figsize'] == (10.0, 10.0)

In [2]:
def inthist(foo, show=False):
    """Create a histogram of integer values.

    Parameters
    ----------
    foo : numpy.ndarray
        An array containing integers.
    show : bool, optional
        If ``True``, create a histogram and return the matplotlib.axes.Axes
        instance.

    Returns
    -------
    inthist : mixed
    """
    xmin = min(foo)
    xmax = max(foo)
    x = np.arange( xmin, xmax+1 )
    n = np.zeros( x.shape, dtype=x.dtype )
    for k in range(len(n)):
        n[k] = np.sum( foo == x[k] )
    if show:
        fig = plt.figure(dpi=100)
        ax = fig.add_subplot(111)
        b = ax.bar(x,n,align='center',width=0.5,color='w')
        ax.set_xlim(xmin-1, xmax+1)
        # ax.set_ylim(0, np.ceil(max(n)/10.0)*10.0)
        ax.set_ylim(0, 10**np.ceil(np.log10(max(n))))
        return ax
    else:
        return (n,x)

In [3]:
def sdss_specobjid(plate, fiber, mjd, run2d, line=None, index=None):
    """Convert SDSS spectrum identifiers into CAS-style specObjID.
    Bits are assigned in specObjID thus:
    ===== ========== =============================================================
    Bits  Name       Comment
    ===== ========== =============================================================
    50-63 Plate ID   14 bits
    38-49 Fiber ID   12 bits
    24-37 MJD        Date plate was observed minus 50000 (14 bits)
    10-23 run2d      Spectroscopic reduction version
    0-9   line/index 0 for use in SpecObj files see below for other uses (10 bits)
    ===== ========== =============================================================
    Parameters
    ----------
    plate, fiber, mjd : :class:`int` or array of int
        Plate, fiber ID, and MJD for a spectrum.  If arrays are
        passed, all must have the same length.  The MJD value must be
        greater than 50000.
    run2d : :class:`int`, :class:`str` or array of int or str
        The run2d value must be an integer or a string of the form 'vN_M_P'.
        If an array is passed, it must have the same length as the other
        inputs listed above.  If the string form is used, the values are
        restricted to :math:`5 \le N \le 6`, :math:`0 \le M \le 99`,
        :math:`0 \le P \le 99`.
    line : :class:`int`, optional
        A line index, only used for defining specObjID for SpecLine files.
        `line` and `index` cannot both be non-zero.
    index : :class:`int`, optional
        An index measure, only used for defining specObjID for SpecLineIndex
        files. `line` and `index` cannot both be non-zero.
    Returns
    -------
    :class:`numpy.ndarray` of :class:`numpy.uint64`
        The specObjIDs of the objects.
    Raises
    ------
    :exc:`ValueError`
        If the sizes of the arrays don't match or if the array values are
        out of bounds.
    Notes
    -----
    * On 32-bit systems, makes sure to explicitly declare all inputs as
      64-bit integers.
    * This function defines the SDSS-III/IV version of specObjID, used for
      SDSS DR8 and subsequent data releases.  It is not compatible with
      SDSS DR7 or earlier.
    * If the string form of `run2d` is used, the bits are assigned by
      the formula :math:`(N - 5) \\times 10000 + M \\times 100 + P`.
    Examples
    --------
    >>> from pydl.pydlutils.sdss import sdss_specobjid
    >>> print(sdss_specobjid(4055,408,55359,'v5_7_0'))
    [4565636362342690816]
    """
    if line is not None and index is not None:
        raise ValueError("line and index inputs cannot both be non-zero!")
    if isinstance(plate, int):
        plate = np.array([plate], dtype=np.uint64)
    if isinstance(fiber, int):
        fiber = np.array([fiber], dtype=np.uint64)
    if isinstance(mjd, int):
        mjd = np.array([mjd], dtype=np.uint64) - 50000
    if isinstance(run2d, str):
        try:
            run2d = np.array([int(run2d)], dtype=np.uint64)
        except ValueError:
            # Try a "vN_M_P" string.
            m = re.match(r'v(\d+)_(\d+)_(\d+)', run2d)
            if m is None:
                raise ValueError("Could not extract integer run2d value!")
            else:
                N, M, P = m.groups()
            run2d = np.array([(int(N) - 5)*10000 + int(M) * 100 + int(P)],
                             dtype=np.uint64)
    elif isinstance(run2d, int):
        run2d = np.array([run2d], dtype=np.uint64)
    if line is None:
        line = np.zeros(plate.shape, dtype=plate.dtype)
    else:
        if isinstance(line, int):
            line = np.array([line], dtype=np.uint64)
    if index is None:
        index = np.zeros(plate.shape, dtype=plate.dtype)
    else:
        if isinstance(index, int):
            index = np.array([index], dtype=np.uint64)
    #
    # Check that all inputs have the same shape.
    #
    if plate.shape != fiber.shape:
        raise ValueError("fiber.shape does not match plate.shape!")
    if plate.shape != mjd.shape:
        raise ValueError("mjd.shape does not match plate.shape!")
    if plate.shape != run2d.shape:
        raise ValueError("run2d.shape does not match plate.shape!")
    if plate.shape != line.shape:
        raise ValueError("line.shape does not match plate.shape!")
    if plate.shape != index.shape:
        raise ValueError("index.shape does not match plate.shape!")
    #
    # Check ranges of parameters
    #
    if ((plate < 0) | (plate >= 2**14)).any():
        raise ValueError("plate values are out-of-bounds!")
    if ((fiber < 0) | (fiber >= 2**12)).any():
        raise ValueError("fiber values are out-of-bounds!")
    if ((mjd < 0) | (mjd >= 2**14)).any():
        raise ValueError("MJD values are out-of-bounds!")
    if ((run2d < 0) | (run2d >= 2**14)).any():
        raise ValueError("MJD values are out-of-bounds!")
    if ((line < 0) | (line >= 2**10)).any():
        raise ValueError("line values are out-of-bounds!")
    if ((index < 0) | (index >= 2**10)).any():
        raise ValueError("index values are out-of-bounds!")
    #
    # Compute the specObjID
    #
    specObjID = ((plate << 50) |
                 (fiber << 38) |
                 (mjd << 24) |
                 (run2d << 10) |
                 (line | index))
    return specObjID

In [4]:
def unwrap_specobjid(specObjID, run2d_integer=False, specLineIndex=False):
    """Unwrap CAS-style specObjID into plate, fiber, mjd, run2d.
    See :func:`~pydl.pydlutils.sdss.sdss_specobjid` for details on how the
    bits within a specObjID are assigned.
    Parameters
    ----------
    specObjID : :class:`numpy.ndarray`
        An array containing 64-bit integers or strings.  If strings are passed,
        they will be converted to integers internally.
    run2d_integer : :class:`bool`, optional
        If ``True``, do *not* attempt to convert the encoded run2d values
        to a string.
    specLineIndex : :class:`bool`, optional
        If ``True`` interpret any low-order bits as being an 'index'
        rather than a 'line'.
    Returns
    -------
    :class:`numpy.recarray`
        A record array with the same length as `specObjID`, with the columns
        'plate', 'fiber', 'mjd', 'run2d', 'line'.
    Examples
    --------
    >>> from numpy import array, uint64
    >>> from pydl.pydlutils.sdss import unwrap_specobjid
    >>> unwrap_specobjid(array([4565636362342690816], dtype=uint64))
    rec.array([(4055, 408, 55359, 'v5_7_0', 0)],
              dtype=[('plate', '<i4'), ('fiber', '<i4'), ('mjd', '<i4'), ('run2d', '<U8'), ('line', '<i4')])
    """
    if (specObjID.dtype.type is np.string_ or
        specObjID.dtype.type is np.unicode_):
        tempobjid = specObjID.astype(np.uint64)
    elif specObjID.dtype.type is np.uint64:
        tempobjid = specObjID.copy()
    else:
        raise ValueError('Unrecognized type for specObjID!')
    run2d_dtype = 'U8'
    if run2d_integer:
        run2d_dtype = 'i4'
    line = 'line'
    if specLineIndex:
        line = 'index'
    unwrap = np.recarray(specObjID.shape,
                         dtype=[('plate', 'i4'), ('fiber', 'i4'),
                                ('mjd', 'i4'), ('run2d', run2d_dtype),
                                (line, 'i4')])
    unwrap.plate = np.bitwise_and(tempobjid >> 50, 2**14 - 1)
    unwrap.fiber = np.bitwise_and(tempobjid >> 38, 2**12 - 1)
    unwrap.mjd = np.bitwise_and(tempobjid >> 24, 2**14 - 1) + 50000
    run2d = np.bitwise_and(tempobjid >> 10, 2**14 - 1)
    if run2d_integer:
        unwrap.run2d = run2d
    else:
        N = ((run2d // 10000) + 5).tolist()
        M = ((run2d % 10000) // 100).tolist()
        P = (run2d % 100).tolist()
        unwrap.run2d = np.array(['v{0:d}_{1:d}_{2:d}'.format(n, m, p)
                                 for n, m, p in zip(N, M, P)])
        for r in (26, 103, 104):
            if (run2d == r).any():
                unwrap.run2d[run2d == r] = str(r)
    unwrap[line] = np.bitwise_and(tempobjid, 2**10 - 1)
    return unwrap

## DR16Q

### Overview

* [data model](https://data.sdss.org/datamodel/files/BOSS_QSO/DR16Q/DR16Q_v4.html)
* [data model (superset)](https://data.sdss.org/datamodel/files/BOSS_QSO/DR16Q/DR16Q_Superset_v3.html)
* [Lyke et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020ApJS..250....8L/abstract)

### Preprocessing Plan

* Compute `specObjID`; add `run2d` column.
* Delete photometric information.
* Move duplicate information to separate table.
* Coerce some matching flags to integer.
* Coerce floating point values (except `RA`, `DEC`) to 32-bit float.

### Known Problems

* The columns `GALEX_MATCHED`, `UKIDSS_MATCHED` are stored as float when they should be integers.  The also contain NaN values, and `GALEX_MATCHED` contains some values which are 2.  None of these cases are described in the DR14Q paper.  Only `FIRST_MATCHED` has valid values and is consistent with its description in the paper.
* The photometric id values, `RUN_NUMBER`, `COL_NUMBER`, are all nonsense values.  All identically zero in fact.
* The duplicate column contains extraneous zeroes, and not all duplicates are actually stored in the column.  See below for examples.  It would be much better if the duplicates were a separate table, mapping primary specObjID to duplicate specObjID.
* `TUNIT` columns should be blank when the value has no unit, instead of `-`.
* I would recommend checking that all units in the table conform to the FITS standard.  For example 'Vega' is a description of a unit, not a physical unit. Also, the `RASS_COUNTS` column has units counts/s, which is misleading, because the units are actually log(counts/s).  Exception: nanomaggies are not standard, but are OK.
* Column names, *e.g.* `FLUX_0.2_2.0keV` contain characters that could be interpreted as integers.
* Not all "primary" objects in DR14Q correspond to real objects in the specObjAll table.  And not all "duplicates" correspond to real objects either.  In these cases there are spectra that exist on disk, but are not now, and probably never will be added to CAS.  This makes it particularly difficult to cleanly join DR14Q to specObjAll.

In [5]:
qso_version = 'v4'
vopath = f"{sas}eboss/qso/DR{release:d}Q/DR{release:d}Q_{qso_version}.fits"
with BytesIO(sc.get(vopath, mode='binary')) as q:
    with fits.open(q) as hdulist:
        dr16q = hdulist['CATALOG'].data
qso_superset_version = 'v3'
vopath_superset = f"{sas}eboss/qso/DR{release:d}Q/DR{release:d}Q_superset_{qso_superset_version}.fits"
with BytesIO(sc.get(vopath_superset, mode='binary')) as q:
    with fits.open(q) as hdulist:
        dr16q_superset = hdulist['CATALOG'].data

### specObjID

Compute `specObjID` for the spectra.  Be careful, because there are some SEGUE spectra in the mix!  Also `dr16q_superset` contains objects that were targeted but never observed, so they have no valid `specobjid` value.

In [6]:
run2d = np.array([boss_run2d]*len(dr16q))
run2d_lookup = dict()
for p in np.unique(dr16q['PLATE']):
    q = f"SELECT plateid, plate, mjd, run2d FROM sdss_dr{release:d}.platex WHERE plate = {p:d}"
    response = qc.query(sql=q, fmt='csv', timeout=600)
    ww = dr16q['PLATE'] == p
    r = response.split('\n')[1].split(',')[-1]
    run2d[ww] = r
    run2d_lookup[int(p)] = r

In [7]:
run2d_superset = np.array([boss_run2d]*len(dr16q_superset))
run2d_superset_lookup = dict()
for p in np.unique(dr16q_superset['PLATE']):
    q = f"SELECT plateid, plate, mjd, run2d FROM sdss_dr{release:d}.platex WHERE plate = {p:d}"
    response = qc.query(sql=q, fmt='csv', timeout=600)
    ww = dr16q_superset['PLATE'] == p
    r = response.split('\n')[1].split(',')[-1]
    run2d_superset[ww] = r
    run2d_superset_lookup[int(p)] = r

In [23]:
# specobjid = np.zeros(dr16q.shape, dtype=np.uint64)
w = run2d != boss_run2d
irun2d = np.zeros(dr16q.shape, dtype=np.uint64)
irun2d[w] = [int(x) for x in run2d[w]]
w = run2d == boss_run2d
irun2d[w] = boss_run2d_int
specobjid = sdss_specobjid(dr16q['PLATE'].astype(np.uint64),
                           dr16q['FIBERID'].astype(np.uint64),
                           dr16q['MJD'].astype(np.uint64) - 50000,
                           irun2d)

In [24]:
w = run2d_superset != boss_run2d
irun2d_superset = np.zeros(dr16q_superset.shape, dtype=np.uint64)
irun2d_superset[w] = [int(x) for x in run2d_superset[w]]
w = run2d_superset == boss_run2d
irun2d_superset[w] = boss_run2d_int
specobjid_superset = sdss_specobjid(dr16q_superset['PLATE'].astype(np.uint64),
                                    dr16q_superset['FIBERID'].astype(np.uint64),
                                    dr16q_superset['MJD'].astype(np.uint64) - 50000,
                                    irun2d_superset)

### Photometric information

These columns contain no useful information at all, so drop them.  Get information by joining on `specobj` instead.

In [26]:
objid = np.zeros(len(dr16q), dtype=np.uint64)
w = dr16q['OBJID'] != ''
objid[w] = np.array(list(map(int, dr16q['OBJID'][w].tolist())), dtype=np.uint64)

In [27]:
w = dr16q['SKYVERSION'] != 2
dr16q['SKYVERSION'][w], dr16q['OBJID'][w], dr16q['RERUN_NUMBER'][w], dr16q['RUN_NUMBER'][w], dr16q['CAMCOL_NUMBER'][w], dr16q['FIELD_NUMBER'][w], dr16q['ID_NUMBER'][w]
# assert (dr16q['RERUN_NUMBER'] == '-').all()
# assert (dr16q['COL_NUMBER'] == 0).all()
# assert (dr16q['FIELD_NUMBER'] == 0).all()
# assert (dr16q['OBJ_ID'] == '-').all()

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [28]:
w = (dr16q['OBJID'] == '') & (dr16q['SKYVERSION'] != 0)
dr16q['SKYVERSION'][w], dr16q['OBJID'][w], dr16q['RERUN_NUMBER'][w], dr16q['RUN_NUMBER'][w], dr16q['CAMCOL_NUMBER'][w], dr16q['FIELD_NUMBER'][w], dr16q['ID_NUMBER'][w]

(array([2, 2], dtype=uint8),
 chararray(['', ''], dtype='<U19'),
 chararray(['301', '301'], dtype='<U3'),
 array([1729, 4836], dtype=int32),
 array([5, 5], dtype=int32),
 array([365, 251], dtype=int32),
 array([105,  25], dtype=int32))

### Targeting flags

Make sure these don't require any special `np.uint64` handling.

In [25]:
assert (dr16q['BOSS_TARGET1'] < 9223372036854775808).all()
assert (dr16q['ANCILLARY_TARGET1'] < 9223372036854775808).all()
assert (dr16q['ANCILLARY_TARGET2'] < 9223372036854775808).all()
assert (dr16q['EBOSS_TARGET0'] < 9223372036854775808).all()
assert (dr16q['EBOSS_TARGET1'] < 9223372036854775808).all()
assert (dr16q['EBOSS_TARGET2'] < 9223372036854775808).all()

### Duplicates

Move the duplicate information to a separate table. Some duplicates are disk only; they do not exist in the `specobjall` table.  We will load all duplicates unflagged, then use a database query to flag the duplicates that do not exist in the database.

In [30]:
dr16q['NSPEC'].max()

73

In [29]:
dr16q_superset['NSPEC'].max()

73

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10, 10)
n, x = inthist(dr16q['NSPEC'])
w = n > 0
fig = plt.figure(dpi=100)
ax = fig.add_subplot(1, 1, 1)
b = ax.bar(x[w], n[w], align='center', width=0.5, color='k')
foo = ax.set_yscale('log')
foo = ax.set_xlim(dr16q['NSPEC'].min()-1, dr16q['NSPEC'].max()+1)
# foo = ax.set_ylim(0, np.ceil(np.log10(max(n))))
foo = ax.set_title(f'Number of DR{release:d}Q Duplicates [{qso_version}]')
foo = ax.set_xlabel('Number of Duplicates [NSPEC]')
foo = ax.set_ylabel('N')

In [ ]:
w = dr16q['NSPEC'] == 73
dr16q['PLATE_DUPLICATE'][w, :]

In [25]:
with open('dr16q_duplicates.csv', 'w', newline='') as csvfile:
    dw = csv.writer(csvfile)
    dw.writerow(['specobjid', 'dupspecobjid', 'run2d', 'plate', 'mjd', 'fiberid', 'disk_only'])
    for k in range(len(dr16q)):
        if dr16q[k]['NSPEC'] > 0:
            w = dr16q['PLATE_DUPLICATE'][k, :] > 0
            Nd = w.sum()
            irun2d = np.zeros((Nd,), dtype=np.uint64)
            plates = dr16q['PLATE_DUPLICATE'][k, w]
            mjds = dr16q['MJD_DUPLICATE'][k, w]
            fibers = dr16q['FIBERID_DUPLICATE'][k, w]
            for i, p in enumerate(plates):
                try:
                    foo = run2d_lookup[p]
                except KeyError:
                    q = f"SELECT run2d FROM sdss_dr{release}.platex WHERE plate = {p:d} AND mjd = {mjds[i]:d}"
                    try:
                        vot = qc.query(sql=q, fmt='votable', timeout=600)
                    except qc.queryClientError as e:
                        if str(e) == 'Error: SQL query did not return any records':
                            q = f"SELECT run2d FROM sdss_dr{release:d}.platex WHERE plate = {p:d}"
                            vot = qc.query(sql=q, fmt='votable', timeout=600)
                        else:
                            raise
                    result = parse_single_table(BytesIO(vot.encode('utf-8')))
                    foo = run2d_lookup[p] = result.array['run2d'][0]
                if foo == boss_run2d:
                    irun2d[i] = boss_run2d_int
                else:
                    irun2d[i] = int(foo)
            dupspecobjid = sdss_specobjid(plates.astype(np.uint64),
                                          fibers.astype(np.uint64),
                                          mjds.astype(np.uint64) - 50000,
                                          irun2d)
            for l in range(Nd):
                dw.writerow([specobjid[k].astype(np.int64),
                             dupspecobjid[l].astype(np.int64),
                             run2d_lookup[plates[l]],
                             plates[l],
                             mjds[l],
                             fibers[l],
                             'FALSE'])

In [26]:
with open('dr16q_superset_duplicates.csv', 'w', newline='') as csvfile:
    dw = csv.writer(csvfile)
    dw.writerow(['specobjid', 'dupspecobjid', 'run2d', 'plate', 'mjd', 'fiberid', 'disk_only'])
    for k in range(len(dr16q_superset)):
        if dr16q_superset[k]['NSPEC'] > 0:
            w = dr16q_superset['PLATE_DUPLICATE'][k, :] > 0
            Nd = w.sum()
            irun2d = np.zeros((Nd,), dtype=np.uint64)
            plates = dr16q_superset['PLATE_DUPLICATE'][k, w]
            mjds = dr16q_superset['MJD_DUPLICATE'][k, w]
            fibers = dr16q_superset['FIBERID_DUPLICATE'][k, w]
            for i, p in enumerate(plates):
                try:
                    foo = run2d_superset_lookup[p]
                except KeyError:
                    q = f"SELECT run2d FROM sdss_dr{release}.platex WHERE plate = {p:d} AND mjd = {mjds[i]:d}"
                    try:
                        vot = qc.query(sql=q, fmt='votable', timeout=600)
                    except qc.queryClientError as e:
                        if str(e) == 'Error: SQL query did not return any records':
                            q = f"SELECT run2d FROM sdss_dr{release:d}.platex WHERE plate = {p:d}"
                            vot = qc.query(sql=q, fmt='votable', timeout=600)
                        else:
                            raise
                    result = parse_single_table(BytesIO(vot.encode('utf-8')))
                    foo = run2d_superset_lookup[p] = result.array['run2d'][0]
                if foo == boss_run2d:
                    irun2d[i] = boss_run2d_int
                else:
                    irun2d[i] = int(foo)
            dupspecobjid = sdss_specobjid(plates.astype(np.uint64),
                                          fibers.astype(np.uint64),
                                          mjds.astype(np.uint64) - 50000,
                                          irun2d)
            for l in range(Nd):
                dw.writerow([specobjid_superset[k].astype(np.int64),
                             dupspecobjid[l].astype(np.int64),
                             run2d_superset_lookup[plates[l]],
                             plates[l],
                             mjds[l],
                             fibers[l],
                             'FALSE'])

In [27]:
4857325810397499392 in specobjid_superset

True

In [28]:
unwrap_specobjid(np.array([4857325810397499392], dtype=np.uint64))

rec.array([(4314, 704, 55855, 'v5_13_0', 0)],
          dtype=[('plate', '<i4'), ('fiber', '<i4'), ('mjd', '<i4'), ('run2d', '<U8'), ('line', '<i4')])

In [ ]:
t = Table(dr16q)
for c in dr16q.columns.names:
    if dr16q[c].dtype.type is np.float64 and c != 'RA' and c != 'DEC':
        # print(c)
        t[c] = dr16q[c].astype(np.float32)
w = dr16q['OBJID'] == ''
t['OBJID'][w] = '0'
t.remove_columns(['PLATE_DUPLICATE', 'MJD_DUPLICATE', 'FIBERID_DUPLICATE', 'SPECTRO_DUPLICATE',
                  'SKYVERSION', 'RUN_NUMBER', 'RERUN_NUMBER', 'CAMCOL_NUMBER', 'FIELD_NUMBER', 'ID_NUMBER'])
t.add_column(Column(name='SPECOBJID', data=np.array(list(map(str, specobjid.tolist())))), index=0)
t.add_column(Column(name='RUN2D', data=run2d), index=4)
t.add_column(Column(name='DISK_ONLY', data=np.zeros((len(dr16q),), dtype=bool)))

In [34]:
ts = Table(dr16q_superset)
for c in dr16q_superset.columns.names:
    if dr16q_superset[c].dtype.type is np.float64 and c != 'RA' and c != 'DEC':
        # print(c)
        ts[c] = dr16q_superset[c].astype(np.float32)
w = dr16q_superset['OBJID'] == ''
ts['OBJID'][w] = '0'
ts.remove_columns(['PLATE_DUPLICATE', 'MJD_DUPLICATE', 'FIBERID_DUPLICATE', 'SPECTRO_DUPLICATE',
                   'SKYVERSION', 'RUN_NUMBER', 'RERUN_NUMBER', 'CAMCOL_NUMBER', 'FIELD_NUMBER', 'ID_NUMBER'])
ts.add_column(Column(name='SPECOBJID', data=np.array(list(map(str, specobjid_superset.tolist())))), index=0)
ts.add_column(Column(name='RUN2D', data=run2d_superset), index=4)
ts.add_column(Column(name='DISK_ONLY', data=np.zeros((len(dr16q_superset),), dtype=bool)))

In [ ]:
t

In [35]:
ts

SPECOBJID,SDSS_NAME,RA,DEC,RUN2D,PLATE,MJD,FIBERID,AUTOCLASS_PQN,AUTOCLASS_DR14Q,IS_QSO_QN,Z_QN,RANDOM_SELECT,Z_10K,Z_CONF_10K,PIPE_CORR_10K,IS_QSO_10K,PRIM_REC,THING_ID,Z_VI,Z_CONF,CLASS_PERSON,Z_DR12Q,IS_QSO_DR12Q,Z_DR7Q_SCH,IS_QSO_DR7Q,Z_DR6Q_HW,Z_DR7Q_HW,IS_QSO_FINAL,Z,SOURCE_Z,Z_PIPE,ZWARNING,OBJID,Z_PCA,ZWARN_PCA,DELTACHI2_PCA,Z_HALPHA,ZWARN_HALPHA,DELTACHI2_HALPHA,Z_HBETA,ZWARN_HBETA,DELTACHI2_HBETA,Z_MGII,ZWARN_MGII,DELTACHI2_MGII,Z_CIII,ZWARN_CIII,DELTACHI2_CIII,Z_CIV,ZWARN_CIV,DELTACHI2_CIV,Z_LYA,ZWARN_LYA,DELTACHI2_LYA,Z_DLA [5],NHI_DLA [5],CONF_DLA [5],BAL_PROB,BI_CIV,ERR_BI_CIV,AI_CIV,ERR_AI_CIV,BI_SIIV,ERR_BI_SIIV,AI_SIIV,ERR_AI_SIIV,BOSS_TARGET1,EBOSS_TARGET0,EBOSS_TARGET1,EBOSS_TARGET2,ANCILLARY_TARGET1,ANCILLARY_TARGET2,NSPEC_SDSS,NSPEC_BOSS,NSPEC,LAMBDA_EFF,ZOFFSET,XFOCAL,YFOCAL,CHUNK,TILE,PLATESN2,PSFFLUX [5],PSFFLUX_IVAR [5],PSFMAG [5],PSFMAGERR [5],EXTINCTION [5],SN_MEDIAN_ALL,DISK_ONLY
str20,str18,float64,float64,str7,int32,int32,int16,str6,str6,int16,float32,int16,float32,int16,int16,int16,int16,int64,float32,int16,int16,float32,int16,float32,int16,float32,float32,int16,float32,str12,float32,int32,str19,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,int64,int64,int64,int64,int64,int32,int32,int32,float32,float32,float32,float32,str14,int32,float32,float32,float32,float32,float32,float32,float32,bool
12699048555652075520,000000.04+064602.3,0.00018665679641571842,6.76732079016156,v5_13_0,11279,58449,85,STAR,STAR,0,2.029856,0,-1.0,-1,-1,-1,0,171139738,-1.0,0,0,-1.0,0,-1.0,-1,-1.0,-1.0,0,-0.00072212715,PIPE,-0.00072212715,0,1237669516904956169,-0.0010385384,0,1132.9832,-0.00653287,0,20.707825,0.006424695,0,19.325714,-1.0,7682,0.0,-1.0,7682,0.0,-1.0,7682,0.0,-1.0,7682,0.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,2048,0,0,0,0,0,-1,4000.0,0.0,223.01915,-30.650879,eboss26,17451,15.2289,2.2861543 .. 16.25594,17.657251 .. 2.0982685,21.598186 .. 19.470226,0.11218224 .. 0.04591866,0.22029035 .. 0.06563499,8.410045,False
9840548378089639936,000000.04+064602.3,0.00018665679641571842,6.76732079016156,v5_13_0,8740,57367,666,STAR,STAR,0,2.052055,0,-1.0,-1,-1,-1,1,171139738,-1.0,0,0,-1.0,0,-1.0,-1,-1.0,-1.0,0,-0.00068493217,PIPE,-0.00068493217,0,1237669516904956169,-0.0012602804,0,1061.7727,-0.013002711,0,20.092707,-0.0045397757,0,45.078735,-1.0,7682,0.0,-1.0,7682,0.0,-1.0,7682,0.0,-1.0,7682,0.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,2048,0,0,0,0,0,-1,4000.0,0.0,-80.42299,210.67645,eboss9,16078,18.6024,2.2861543 .. 16.25594,17.657251 .. 2.0982685,21.598186 .. 19.470226,0.11218224 .. 0.04591866,0.22029035 .. 0.06563499,10.498864,False
8663957691486720000,000000.05+272811.6,0.00021481019643943,27.469903330451316,v5_13_0,7695,57654,574,STAR,STAR,0,2.120619,0,-1.0,-1,-1,-1,1,349500180,-1.0,0,0,-1.0,0,-1.0,-1,-1.0,-1.0,0,-0.0008394882,PIPE,-0.0008394882,0,1237663233379074508,-0.00069412077,0,71.91638,-0.00832472,0,20.500704,-0.011810769,0,25.309147,-1.0,7682,0.0,-1.0,7682,0.0,-1.0,7682,0.0,-1.0,7682,0.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,2048,0,0,0,0,0,-1,4000.0,0.0,-216.4216,21.200415,eboss1,16242,12.4032,1.9757667 .. 6.4316287,21.576687 .. 2.0363147,21.75525 .. 20.465101,0.11713276 .. 0.115286015,0.25246164 .. 0.07522035,4.093675,False
8838397398546141184,000000.06-021415.2,0.0002794093173577039,-2.237564113414122,v5_13_0,7850,56956,302,STAR,STAR,0,1.282475,0,-1.0,-1,-1,-1,1,60588214,-1.0,0,0,-1.0,0,-1.0,-1,-1.0,-1.0,0,-5.8219717e-05,PIPE,-5.8219717e-05,0,1237678881020641645,0.033841155,32,657.13165,-0.0034867097,0,14.696323,0.015614027,0,17.635715,-1.0,7682,0.0,-1.0,7682,0.0,-1.0,7682,0.0,-1.0,7682,0.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1

In [ ]:
t.write('dr16q_datalab.fits')

In [36]:
ts.write('dr16q_superset_datalab.fits')

## ELG_CLASSIFICATION

In [5]:
elg_version = 'v1_0'
vopath = f"{sas}eboss/elg/elg_classifier/{elg_version}/eboss-elg-classification_{elg_version}.fits"
with BytesIO(sc.get(vopath, mode='binary')) as q:
    with fits.open(q) as hdulist:
        elg = hdulist[1].data

In [6]:
t = Table(elg)
t['MJD'] = t['MJD'].astype(np.int32)
t['PLATE'] = t['PLATE'].astype(np.int16)
t['FIBERID'] = t['FIBERID'].astype(np.int16)
t['TYPE'] = t['TYPE'].astype(np.int16)
m = re.match(r'v(\d+)_(\d+)_(\d+)', boss_run2d)
N, M, P = m.groups()
br = (int(N) - 5)*10000 + int(M) * 100 + int(P)
specobjid = sdss_specobjid(t['PLATE'].astype(np.uint64), t['FIBERID'].astype(np.uint64), t['MJD'].astype(np.uint64) - 50000, np.array([br]*len(t), dtype=np.uint64)).astype(np.int64)

In [7]:
for i in range(len(t)):
    q = f"SELECT run2d, plate, mjd, fiberid FROM sdss_dr{release:d}.specobjall WHERE specobjid = {specobjid[i]:d}"
    # print(q)
    # print(t['PLATE'][i], t['MJD'][i], t['FIBERID'][i])
    result = qc.query(sql=q, fmt='csv', timeout=600)
    # print(result)
    run2d, plate, mjd, fiberid = result.split('\n')[1].strip().split(',')
    assert run2d == 'v5_13_0'
    assert int(plate) == t['PLATE'][i]
    assert int(mjd) == t['MJD'][i]
    assert int(fiberid) == t['FIBERID'][i]

In [8]:
(np.unique(specobjid) == specobjid).all()

False

In [11]:
sorted(np.unique(specobjid)) == sorted(specobjid)

True

In [12]:
t.add_column(specobjid, name='SPECOBJID', index=0)

In [13]:
t

SPECOBJID,MJD,PLATE,FIBERID,z,O3_Hb,O2_Hb,sigma_o3,sigma_star,u_g,g_r,r_i,i_z,TYPE
int64,int32,int16,int16,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16
4303371506141122560,55544,3822,662,0.45585498,-0.10631379554675359,0.15280902158392662,2.113111818225317,2.060860254999036,2.091821654939249,1.634591133746774,0.6878215914106747,0.39862390589964036,2
5821040883685150720,56063,5170,503,0.54958546,0.6837303839527116,0.5177330993970278,2.188220189200298,2.385730824976833,1.2792134812061065,1.3173727002840714,0.9219287534665952,0.5378095733832176,3
7292484314855657472,56365,6477,111,0.6770768,0.2729628332762776,0.263795127500976,2.262145362235989,2.327437325129091,0.5588234458717807,0.8927006111782845,0.7387906117517886,0.15757234002977327,2
6816383680770232320,56089,6054,675,0.58961457,1.0332131458994422,0.185912623939589,2.1714848778020044,2.5236040234704302,2.0272985089751216,1.6268764459152294,1.0814204705839856,0.4669580317397113,3
4533082846131408896,55325,4026,763,0.40344292,0.8696019555200147,0.5817793621251371,2.076130840783014,2.1402917218335835,1.797855119813665,1.595616693681972,0.5705848001887368,0.4009193508476123,3
5130795513344708608,55588,4557,253,0.62240684,-0.19225856584015572,0.08855782218296977,2.1339867670859065,1.7936856735837574,0.9851932622407382,0.8941441080175316,0.7543408437258243,0.3446281437224066,2
-8499255819306250240,57427,8835,591,0.7634385,0.4590402472950636,0.34106652992150766,2.1576762172615513,2.929418925714293,0.25843176076870833,0.32062432189562884,0.5520613131803422,0.2531994720201034,3
5130800461147033600,55588,4557,271,0.668348,-0.03672907337288249,0.15593071424910263,2.231084365580252,2.3942452893773534,0.8906361149184541,0.774856111624743,0.7356713969785353,0.29393535702795504,2
5130800736024940544,55588,4557,272,0.68234134,-0.3116326936535396,0.17148538601105226,2.4098020547728876,2.056633271435047,0.3866827032072173,0.7096548467282418,0.7116195342910494,0.23993540184093476,2


In [14]:
t.write('elg_classification_datalab.fits')